<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/pytorch/blob/main/convolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!git clone https://github.com/NVIDIA/PyProf.git

Cloning into 'PyProf'...
remote: Enumerating objects: 1480, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 1480 (delta 51), reused 67 (delta 44), pack-reused 1390
Receiving objects: 100% (1480/1480), 352.34 KiB | 19.57 MiB/s, done.
Resolving deltas: 100% (1036/1036), done.


In [16]:
!cd PyProf;pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/PyProf
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for nvidia-pyprof: filename=nvidia_pyprof-3.11.0-py3-none-any.whl size=86612 sha256=28bbab74beef72aacd13aecd68f76f61ad8f4589e8b17caf0ef929f96acfa088
  Stored in directory: /tmp/pip-ephem-wheel-cache-q208iqde/wheels/0b/e4/23/3a541864cf1d361dcf0c6fb513cee71a81c68d494e720da3e3
Successfully built nvidia-pyprof


see https://github.com/NVIDIA/PyProf how to use

In [1]:
import torch

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision as vision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [4]:
dataset_train=vision.datasets.CIFAR10(".",download=True,train=True,transform=transform)
dataset_test=vision.datasets.CIFAR10(".",download=True,train=False,transform=transform)
loader_train=DataLoader(dataset_train,batch_size=512,shuffle=True,num_workers=4)
loader_test=DataLoader(dataset_test,batch_size=512,shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [5]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    # input is (*,3,32,32)
    self.conv1=nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3)
    self.relu=nn.ReLU()
    # input is (*,32,30,30)
    self.pool1=nn.MaxPool2d(kernel_size=(2,2))
    # input is (*,32,15,15)
    self.conv2=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3)
    # input is (*,64,13,13)
    self.pool2=nn.MaxPool2d(kernel_size=(2,2))
    # input is (*,64,6,6)
    self.conv3=nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3)
    # input is (*,64,4,4)
    self.pool3=nn.MaxPool2d(kernel_size=(2,2))
    # input is (*,64,2,2)
    self.flatten=torch.flatten
    # input is (*,64x2x2)
    self.fc1=nn.Linear(in_features=2*2*64,out_features=64)
    self.fc2=nn.Linear(in_features=64,out_features=10)

  def forward(self,x):
    x=self.conv1(x)
    x=self.relu(x)
    x=self.pool1(x)

    x=self.conv2(x)
    x=self.relu(x)
    x=self.pool2(x)
    
    x=self.conv3(x)
    x=self.relu(x)
    x=self.pool3(x)
    
    x=self.flatten(x,start_dim=1)
    x=self.fc1(x)
    x=self.relu(x)
    x=self.fc2(x)
    return x
    

In [6]:
model=Net().cuda()

In [7]:
loss_fn=nn.CrossEntropyLoss()
from torch.optim import SGD,Adam

In [8]:
#optimizer=SGD(model.parameters(),lr=0.005)
optimizer=Adam(model.parameters())

In [9]:
epochs=10
from tqdm import tqdm
for epoch in range(epochs):
  loader_train=tqdm(loader_train)
  for i, (imgs,labels) in enumerate(loader_train,0):
    optimizer.zero_grad()
    imgs=imgs.cuda()
    labels=labels.cuda()
    outputs=model(imgs)
    loss=loss_fn(outputs,labels)
    loss.backward()
    optimizer.step()
  print(loss.item()) 

100%|██████████| 98/98 [00:11<00:00,  8.28it/s]


1.7584096193313599


100%|██████████| 98/98 [00:11<00:00,  8.39it/s]


1.388598084449768


100%|██████████| 98/98 [00:09<00:00, 10.18it/s]


...

1.0854449272155762


100%|██████████| 98/98 [00:09<00:00,  9.88it/s]


1.178902268409729


100%|██████████| 98/98 [00:09<00:00, 10.19it/s]

0.971398115158081


In [10]:
total=0
correct=0
for data in loader_test:
  imgs,labels=data
  imgs=imgs.cuda()
  labels=labels.cuda()
  outputs=model(imgs)
  # the second return value is the index of the max i.e. argmax
  _,predicted=torch.max(outputs.data,1)
  total+=labels.size(0)
  correct+=(predicted==labels).sum()


In [11]:
print(correct/total)

tensor(0.6139, device='cuda:0')
